In [238]:
import pandas as pd
import numpy as np

In [239]:
df = pd.read_csv('OB_UK.txt', sep=',', header=0, parse_dates=['date_t'], 
                 date_parser=lambda d: pd.datetime.strptime(d, "%Y/%m/%d %H") if d!='date_t' else pd.NaT,
                 dtype={'msisdn':str, 'imsi':str}) 
df.drop(df[df['date_t'].isnull()].index, inplace=True)

In [240]:
grouped_df = df.groupby(['msisdn', 'imsi', df['date_t'].dt.date]).size().reset_index(name='hours')

In [254]:
def consecutive_days(arr):
    return (arr.iloc[-1] - arr.iloc[0]).days + 1

def merge_consecutive_date(df):
    sorted_date = df.sort_values(by=['date_t'])[['date_t', 'hours']]

    consecutived = sorted_date['date_t']-pd.to_timedelta(1, unit='D') != sorted_date['date_t'].shift(1)
    #if len(consecutived) > 1:
    #    consecutived.iloc[0] = consecutived.iloc[1]
    consecutived = consecutived.astype(np.int32)

    merged_date = sorted_date.groupby(consecutived.cumsum(),as_index=False).agg({
                                 'date_t':[('start_date', 'first'), ('days', consecutive_days)], 
                                 'hours':[('sum_hours','sum')]})
    merged_date.columns = merged_date.columns.droplevel(0)

    return merged_date

In [255]:
merged_df = grouped_df.groupby(['msisdn', 'imsi']).apply(merge_consecutive_date)
merged_df = merged_df.reset_index(level=2, drop=True)

sum_hours  start_date  days
msisdn       imsi                                        
40773166     420011901830448          2  2018-09-08     1
5.01e+08     420016102108503        172  2018-09-06    10
500000011    420017308101132        183  2018-09-01    11
             420017308101132         75  2018-09-13     5
500000021    420012301889848         98  2018-09-01    14
500000029    420014609060547          9  2018-09-13     1
500000039    420014002418775         28  2018-09-01     2
             420014002418775         41  2018-09-04     2
             420014002418775         82  2018-09-27     4
500000041    420017308476823         25  2018-09-29     2
             420017308476824         15  2018-09-30     1
500000066    420013301768883        232  2018-09-01    16
             420013301768883         44  2018-09-25     3
500000073    420014002893704        160  2018-09-17    12
500000088    420013301715341          7  2018-09-04     3
             420013301715341        251  2018-09-08    23
500000152    420014500567268        271  2018-09-01    16
500000229    420017302978718        289  2018-09-16    15
500000238    420013302634420        101  2018-09-17     8
500000307    420013301803221         58  2018-09-01     4
500000340    420017309092552        179  2018-09-15    13
500000364    420012106908684          8  2018-09-01     1
500000388    420015604251468         32  2018-09-03     7
             420015604251468         24  2018-09-21     4
500000496    420017305038386         13  2018-09-02     1
             420017305038386        324  2018-09-09    22
500000582    420017308661387        156  2018-09-01    11
             420017308661387        235  2018-09-16    15
500000586    420013301099119          3  2018-09-01     1
             420013301099119         10  2018-09-03     3
...                                 ...         ...   ...
831047559195 420014608823117        155  2018-09-11    10
831047565129 420013408112941         24  2018-09-20     2
             420013408112941         16  2018-09-24     2
831047568582 420013408164230         55  2018-09-27     4
831047591683 420013710140073        153  2018-09-01    10
831047664454 420013408250457          8  2018-09-30     1
831047703244 420014608852915          8  2018-09-22     2
831047710204 420014608848212         67  2018-09-11     3
             420014608848212         73  2018-09-16     4
             420014608848212          8  2018-09-23     3
             420014608848212         71  2018-09-27     4
831047728211 420013710321541          1  2018-09-03     1
831047738637 420015603590994         27  2018-09-02     2
831047750800 420012302530796          9  2018-09-21     1
831047763777 420013808047836         24  2018-09-13     2
831047775635 420013807925503          9  2018-09-08     1
831047802906 420016500360008        124  2018-09-16     8
831047820742 420015603769965         26  2018-09-21     3
831047821358 420016500650584          1  2018-09-27     1
831047829656 420012409077627         13  2018-09-20     2
831047913166 420013808096903         49  2018-09-15     3
831047915118 420013709416935         40  2018-09-08     3
831047924191 420014607987636          6  2018-09-22     2
831047960881 420016103348007          4  2018-09-28     1
831047975856 420015604018813         26  2018-09-21     2
831047983101 420010807124860         22  2018-09-23     2
             420010807124860          3  2018-09-30     1
831047985236 420013709200182         57  2018-09-23     4
831047992434 420012409077544          3  2018-09-21     1
831047995409 420013302253102         48  2018-09-25     4

[36643 rows x 3 columns]

In [256]:
merged_df.to_csv(r'OB_UK_merged.csv', sep=',', header=True, index=True, na_rep='',
                 columns=['start_date', 'days', 'sum_hours'])